<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [19]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [21]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.8
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2
#количество входов/выходов соответственно
countX = 2
countY = 2

networkCharact = [layer, neurons, countX, countY, learningRate]

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [22]:
def deltaRule(function, data, level = 1, N = 2, nabla = 0.1):
  si = [0] * N
  yi = [0] * N
  epsi = [0] * N
  level_i = 0
  k = 0

  while(level_i < level): #цикл по слоям
    wi = [[randomNumber() for i in range(N)] for j in range(N)] #синаптические веса

    print("\tЭпоха №" + str(level_i + 1))

    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = ( " + str(data[data.columns[0]][k]) + ", " + str(data[data.columns[1]][k]) + ' )')

      for i in range(N):
        for j in range(N):
          si[i] += data[data.columns[j]][k] * wi[j][i]

        yi[i] = function(si[i])
        epsi[i] = data[data.columns[N + i]][k] - yi[i]
        eps += epsi[i]**2

      rez_i = pd.DataFrame([si, yi, epsi], index=['S', 'Y расч.', 'EPS'], columns=['neuron ' + str(i+1) for i in range(N)])
      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(N):
        for j in range(N):
          wi[j][i] = wi[j][i] - nabla * epsi[i] * data[data.columns[j]][k]

      k += 1
    level_i += 1

  #

In [23]:
table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(hyperbolicTangentFunction, table, layer, neurons, learningRate)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

	Эпоха №1
Матрица весов для 1-й итерации:
          0         1
0  0.399705  0.567412
1  0.197778  0.781624

Для X = ( 0, 0 )
         neuron 1  neuron 2
S             0.0       0.0
Y расч.       0.0       0.0
EPS           1.0       0.0

СКО: 1.0


Матрица весов для 2-й итерации:
          0         1
0  0.399705  0.567412
1  0.197778  0.781624

Для X = ( 0, 1 )
         neuron 1  neuron 2
S        0.197778  0.781624
Y расч.  0.098568  0.372060
EPS     -0.098568 -0.372060

СКО: 0.14814426418877671


Матрица весов для 3-й итерации:
          0         1
0  0.399705  0.567412
1  0.276633  1.079272

Для X = ( 1, 0 )
         neuron 1  neuron 2
S        0.597484  1.349036
Y расч.  0.290161  0.587944
EPS     -0.290161 -0.587944

СКО: 0.4298714260660957


Матрица весов для 4-й итерации:
          0         1
0  0.631834  1.037767
1  0.276633  1.079272

Для X = ( 1, 1 )
         neuron 1  neuron 2
S        1.505951  3.466075
Y расч.  0.636921  0.939414
EPS      0.363079  0.060586

СКО: 0.135

In [ ]:
#1-Количество входов и выходов могут быть различны
#2-Реализовать расчёт промежуточных слоёв

In [ ]:
#Характеристики многослойной сети:
#1- вектор из кол-ва нейронов в каждом слое (в каждом слое м/б разное кол-во нейронов)
#2- кол-во выходов = кол-ву нейронов в крайнем слое
#3- входов может быть сколько угодно
#4- в цикле: wij: i-номер нейрона, j-номер входа
#5- размер s, eps равен количеству выходов